In [26]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json
import os

In [27]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36',
           'Referer':'https://www.motomalaysia.com/'}

In [28]:
with open('motor_models.jsonl') as fopen:
    models = [line.strip().strip('"') for line in fopen]

In [108]:
len(models)
from unidecode import unidecode
# import re

In [78]:
# Define a mapping from Unicode characters to strings
unicode_to_string = {
    '\u2013': '-',
    '\u00a0': ' ',
    '\u2588': '',
}

# Example usage
input_char = '\u2588'  # This would be your input character

# Convert the Unicode character to the corresponding string
output_string = unicode_to_string.get(input_char, input_char)
output_string

''

In [124]:
def crawl(x):
    try:
        url = x
        r = requests.get(url,headers = headers)
        soup = BeautifulSoup(r.content, "html.parser")
    
        name_list = []
        value_list = []
        combined_list = []
        moto_data = []
        overviews = [] 
        
        attr = {}
        model_name = soup.find('h1' ,{'class': 'entry-title'}).text
        attr['model_name'] = model_name
        
        overview = soup.find('div' ,{'class': 'textdesc2'}).find_all('p')
        
        for i in overview:
            overviews.append(i.text)
        attr['overviews'] = overviews
        
        # cost = soup.find('table' ,{'id': 'isipost'}).find_all('td', {'class': 'lapistengah'})
        # print(url + " : " + str(cost))
        # costs = [f"Price : {cost[0].text}", f"Monthly (5 Years) : {cost[1].text}", f"Monthly (7 Years) : {cost[2].text}", f"Monthly (9 Years) : {cost[3].text}"]
        # attr['costs'] = costs
        prices_name = []
        for i in soup.find('table' ,{'id': 'isipost'}).find_all('th'):
            if i.get('id') == 'bghitam':
                continue
            else:
                prices_name.append((i.text).replace('\n', ' '))
                
        prices_value = []
        for i in soup.find('table' ,{'id': 'isipost'}).find_all('td', {'class': 'lapistengah'}):
            prices_value.append(i.text)
    
        combined_list_price = [] 
        for name, value in zip(prices_name, prices_value):
                combined_list_price.append(f"{name}: {value}")
     
        try:
            for i in soup.find('table', {'class': 'specscsstable varianmodel'}).find_all('td', {'class': 'lapissatu'}):
                values = unidecode(i.text)
                values = values.replace('### -', '')
                # values = values.replace('\n', '')
                values = values.replace('#', '')
                name_list.append(values)
        except:
            val = soup.find_all('table', {'id': 'isipost'})
            for i in val[1].find_all('td', {'class': 'lapissatu'}):
                values = unidecode(i.text)
                values = values.replace('### -', '')
                # values = values.replace('\n', '')
                values = values.replace('#', '')
                name_list.append(values)
        
        try:
            for i in soup.find('table', {'class': 'specscsstable varianmodel'}).find_all('td'):
                if i.get('class') == None:
                    values = unidecode(i.text)
                    values = values.replace('### -', '')
                    # values = values.replace('\n', '')
                    values = values.replace('#', '')
                    value_list.append(values)
        except:
            val = soup.find_all('table', {'id': 'isipost'})
            for i in val[1].find_all('td'):
                if i.get('class') == None:
                    values = unidecode(i.text)
                    values = values.replace('### -', '')
                    # values = values.replace('\n', '')
                    values = values.replace('#', '')
                    value_list.append(values)
        
        for name, value in zip(name_list, value_list):
            combined_list.append(f"{name}: {value}")
        attr['specs'] = combined_list
        
        count=0
        
        for i in soup.find('table', {'id':'kotakgambar'}).find_all('img'):
            img_dict = {}
            url_image = i.get('src')
            img_file = os.path.join('pic', f"{attr['model_name']}_{count}.jpg")
            with open(img_file, 'wb') as fopen:
                r = requests.get(url_image)
                fopen.write(r.content)
        
            img_dict['img_url'] = url_image
            img_dict['local_image'] = img_file
            img_dict['specs'] = attr
            moto_data.append(img_dict)
            count += 1
    
        return moto_data
        
    except Exception as e: 
        print(url)
        print(e)
        return 

In [125]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from tqdm import tqdm

max_workers = 10
urls = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(crawl, x) for x in models]
    
    for future in tqdm(futures, total=len(models)):
        result = future.result()
        if result:
            for x in result:
                with open('motomalaysia-data.jsonl', 'a') as f:
                    json.dump(x, f)
                    f.write('\n')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 224/224 [00:24<00:00,  9.24it/s]


In [84]:
with open('motomalaysia-data.jsonl') as fopen:
    datas = [line.strip().strip('"') for line in fopen]

In [85]:
len(datas)

672